In [1]:
import pandas as pd
import random
from datetime import datetime, timedelta
import os

# Lokasi gudang dan koordinat (latitude, longitude)
location_coords = {
    'Jakarta': (-6.2088, 106.8456),
    'Bandung': (-6.9175, 107.6191),
    'Surabaya': (-7.2504, 112.7688),
    'Yogyakarta': (-7.7956, 110.3695),
    'Semarang': (-6.9667, 110.4167),
    'Medan': (3.5952, 98.6722),
    'Denpasar': (-8.6500, 115.2167),
    'Makassar': (-5.1477, 119.4327),
    'Balikpapan': (-1.2675, 116.8289),
    'Manado': (1.4748, 124.8421)
}

# Buah dan harga kisaran
fruits = {
    'Apel': (20, 30),
    'Pisang': (10, 15),
    'Mangga': (15, 20),
    'Jeruk': (18, 22),
    'Pepaya': (7, 10),
    'Durian': (35, 45),
    'Rambutan': (12, 18)
}

# Kualitas
qualities = ['Bagus', 'Sedang', 'Buruk']

# Generate data
data = []
start_date = datetime(2025, 4, 1)
id_counter = 1

for lokasi, (lat, lon) in location_coords.items():
    for _ in range(400):  # 200 per lokasi
        buah = random.choice(list(fruits.keys()))
        stok = random.randint(200, 1000)
        harga = random.randint(*fruits[buah])
        tanggal = start_date + timedelta(days=random.randint(0, 34))  # 5 minggu
        kualitas = random.choice(qualities)
        data.append([
            id_counter, lokasi, buah, stok,
            tanggal.strftime("%Y-%m-%d"), kualitas, harga, lat, lon
        ])
        id_counter += 1

# Buat DataFrame
df = pd.DataFrame(data, columns=[
    'ID', 'Lokasi', 'Nama_Buah', 'Stok_kg',
    'Tanggal_Update', 'Kualitas', 'Harga_per_kg',
    'Latitude', 'Longitude'
])

# Simpan per lokasi
output_dir = "C:/Users/DELL/Downloads/project_DWH"
os.makedirs(output_dir, exist_ok=True)

for lokasi in df['Lokasi'].unique():
    df_lokasi = df[df['Lokasi'] == lokasi]
    filename = f"{lokasi.replace(' ', '_')}.csv"
    path = os.path.join(output_dir, filename)
    df_lokasi.to_csv(path, index=False)

print("Berhasil menambahkan lat & lon ke data stok buah per lokasi dan simpan sebagai CSV.")


Berhasil menambahkan lat & lon ke data stok buah per lokasi dan simpan sebagai CSV.


In [2]:
import pandas as pd
import mysql.connector
import os

# ========== EXTRACT ==========
def extract(folder_path):
    csv_files = [file for file in os.listdir(folder_path) if file.endswith('.csv')]
    
    df_list = []
    for file in csv_files:
        full_path = os.path.join(folder_path, file)
        df = pd.read_csv(full_path)
        df_list.append(df)
    
    df_all = pd.concat(df_list, ignore_index=True)
    print(f"✅ Extract selesai! Jumlah data: {len(df_all)} baris")
    return df_all

# ========== TRANSFORM ========== 
def transform(df): 
    # Cek nilai null sebelum transformasi 
    print(f"Missing values sebelum transform:\n{df.isnull().sum()}")

    # Rename kolom agar konsisten
    df.rename(columns={ 
        'Nama_Buah': 'produk', 
        'Lokasi': 'lokasi', 
        'Stok_kg': 'stok', 
        'Tanggal_Update': 'tanggal', 
        'Kualitas': 'kualitas', 
        'Harga_per_kg': 'harga', 
        'Latitude': 'lat', 
        'Longitude': 'lon' 
    }, inplace=True)

    # Format kolom tanggal
    df['tanggal'] = pd.to_datetime(df['tanggal'], errors='coerce')

    # Drop baris yang kolom pentingnya null
    df.dropna(subset=['tanggal', 'produk', 'lokasi', 'stok', 'kualitas', 'harga', 'lat', 'lon'], inplace=True)

    # Normalisasi text
    df['produk'] = df['produk'].str.title()
    df['lokasi'] = df['lokasi'].str.title()
    df['kualitas'] = df['kualitas'].str.title()

    # Validasi stok & harga
    df = df[(df['stok'] >= 0) & (df['harga'] > 0)]

    # Reset index
    df.reset_index(drop=True, inplace=True)

    # Cek nilai null setelah transformasi
    print(f"\nMissing values setelah transform:\n{df.isnull().sum()}")

    print(f"\n✅ Transform selesai! Sisa data:  {len(df)} baris")
    return df


# ========== LOAD ==========
def load(df):
    conn = mysql.connector.connect(
        host='localhost',
        user='root',
        password='Adhiestyaresthu08#',
        database='gudang_db'
    )
    cursor = conn.cursor()

    # Drop tabel jika sudah ada
    cursor.execute("DROP TABLE IF EXISTS gudang_stok")

    # Buat ulang tabel dengan struktur lengkap
    cursor.execute("""
    CREATE TABLE gudang_stok (
        id INT AUTO_INCREMENT PRIMARY KEY,
        tanggal DATE,
        produk VARCHAR(100),
        lokasi VARCHAR(50),
        stok INT,
        kualitas VARCHAR(20),
        harga INT,
        lat FLOAT,
        lon FLOAT
    )
    """)


    # Insert data
    for _, row in df.iterrows():
        cursor.execute("""
            INSERT INTO gudang_stok (
                tanggal, produk, lokasi, stok, kualitas, harga, lat, lon
            ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
        """, (
            row['tanggal'].strftime('%Y-%m-%d'),
            row['produk'],
            row['lokasi'],
            int(row['stok']),
            row['kualitas'],
            int(row['harga']),
            float(row['lat']),
            float(row['lon'])
        ))

    conn.commit()
    cursor.close()
    conn.close()
    print("✅ Load ke MySQL selesai!")

# ========== MAIN ==========
def etl_process():
    folder_path = r'C:\Users\DELL\Downloads\project_DWH'
    df = extract(folder_path)
    df_clean = transform(df)
    load(df_clean)

# Jalankan ETL
etl_process()


✅ Extract selesai! Jumlah data: 8000 baris
Missing values sebelum transform:
ID                                                                      4000
Lokasi                                                                  4000
Nama_Buah                                                               4000
Stok_kg                                                                 4000
Tanggal_Update                                                          4000
Kualitas                                                                4000
Harga_per_kg                                                            4000
Latitude                                                                4000
Longitude                                                               4000
id;"tanggal";"produk";"lokasi";"stok";"kualitas";"harga";"lat";"lon"    4000
dtype: int64

Missing values setelah transform:
ID                                                                         0
lokasi                      

In [3]:
import pandas as pd
import mysql.connector
import os

# ========== KONEKSI ==========
def get_connection():
    return mysql.connector.connect(
        host='localhost',
        user='root',
        password='Adhiestyaresthu08#',
        database='gudang_db'
    )

# ========== DIM & FACT SETUP ==========
def setup_star_schema():
    conn = get_connection()
    cursor = conn.cursor()

    # Drop existing tables (hati-hati di produksi!)
    cursor.execute("DROP TABLE IF EXISTS fact_stok")
    cursor.execute("DROP TABLE IF EXISTS dim_tanggal")
    cursor.execute("DROP TABLE IF EXISTS dim_produk")
    cursor.execute("DROP TABLE IF EXISTS dim_lokasi")

    # Create dimensi dan fact
    cursor.execute("""
        CREATE TABLE dim_tanggal (
            id_tanggal INT AUTO_INCREMENT PRIMARY KEY,
            tanggal DATE UNIQUE
        )
    """)
    cursor.execute("""
        CREATE TABLE dim_produk (
            id_produk INT AUTO_INCREMENT PRIMARY KEY,
            produk VARCHAR(100),
            kualitas VARCHAR(20),
            UNIQUE(produk, kualitas)
        )
    """)
    cursor.execute("""
        CREATE TABLE dim_lokasi (
            id_lokasi INT AUTO_INCREMENT PRIMARY KEY,
            lokasi VARCHAR(50),
            lat FLOAT,
            lon FLOAT,
            UNIQUE(lokasi, lat, lon)
        )
    """)
    cursor.execute("""
        CREATE TABLE fact_stok (
            id INT AUTO_INCREMENT PRIMARY KEY,
            id_tanggal INT,
            id_produk INT,
            id_lokasi INT,
            stok INT,
            harga INT,
            FOREIGN KEY (id_tanggal) REFERENCES dim_tanggal(id_tanggal),
            FOREIGN KEY (id_produk) REFERENCES dim_produk(id_produk),
            FOREIGN KEY (id_lokasi) REFERENCES dim_lokasi(id_lokasi)
        )
    """)

    conn.commit()
    cursor.close()
    conn.close()

# ========== EXTRACT ==========
def extract(folder_path):
    csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
    df_list = [pd.read_csv(os.path.join(folder_path, f)) for f in csv_files]
    df_all = pd.concat(df_list, ignore_index=True)
    print(f"✅ Extract selesai! Jumlah data: {len(df_all)} baris")
    return df_all

# ========== TRANSFORM ==========
def transform(df):
    df.rename(columns={
        'Nama_Buah': 'produk',
        'Lokasi': 'lokasi',
        'Stok_kg': 'stok',
        'Tanggal_Update': 'tanggal',
        'Kualitas': 'kualitas',
        'Harga_per_kg': 'harga',
        'Latitude': 'lat',
        'Longitude': 'lon'
    }, inplace=True)
    
    df['tanggal'] = pd.to_datetime(df['tanggal'], errors='coerce')
    df.dropna(subset=['tanggal', 'produk', 'lokasi', 'stok', 'kualitas', 'harga', 'lat', 'lon'], inplace=True)

    df['produk'] = df['produk'].str.title()
    df['lokasi'] = df['lokasi'].str.title()
    df['kualitas'] = df['kualitas'].str.title()

    df = df[(df['stok'] >= 0) & (df['harga'] > 0)]
    df.reset_index(drop=True, inplace=True)

    print(f"✅ Transform selesai! Sisa data: {len(df)} baris")
    return df

# ========== LOAD ==========
def load_to_star(df):
    conn = get_connection()  # Use get_connection() instead of connect()
    cursor = conn.cursor()

    lokasi_cache = {}
    tanggal_cache = {}
    produk_cache = {}

    for _, row in df.iterrows():
        # dim_tanggal
        tanggal = row['tanggal'].date() if hasattr(row['tanggal'], 'date') else row['tanggal']
        if tanggal not in tanggal_cache:
            cursor.execute("SELECT id_tanggal FROM dim_tanggal WHERE tanggal = %s", (tanggal,))
            result = cursor.fetchone()
            if result:
                tanggal_cache[tanggal] = result[0]
            else:
                cursor.execute("INSERT INTO dim_tanggal (tanggal) VALUES (%s)", (tanggal,))
                tanggal_cache[tanggal] = cursor.lastrowid
        id_tanggal = tanggal_cache[tanggal]

        # dim_lokasi
        lokasi_key = f"{row['lokasi']}_{row['lat']}_{row['lon']}"
        if lokasi_key not in lokasi_cache:
            cursor.execute("SELECT id_lokasi FROM dim_lokasi WHERE lokasi = %s AND lat = %s AND lon = %s",
                           (row['lokasi'], row['lat'], row['lon']))
            result = cursor.fetchone()
            if result:
                lokasi_cache[lokasi_key] = result[0]
            else:
                cursor.execute("INSERT INTO dim_lokasi (lokasi, lat, lon) VALUES (%s, %s, %s)",
                               (row['lokasi'], row['lat'], row['lon']))
                lokasi_cache[lokasi_key] = cursor.lastrowid
        id_lokasi = lokasi_cache[lokasi_key]

        # dim_produk
        produk_key = f"{row['produk']}_{row['kualitas']}"
        if produk_key not in produk_cache:
            cursor.execute("SELECT id_produk FROM dim_produk WHERE produk = %s AND kualitas = %s",
                           (row['produk'], row['kualitas']))
            result = cursor.fetchone()
            if result:
                produk_cache[produk_key] = result[0]
            else:
                cursor.execute("INSERT INTO dim_produk (produk, kualitas) VALUES (%s, %s)",
                               (row['produk'], row['kualitas']))
                produk_cache[produk_key] = cursor.lastrowid
        id_produk = produk_cache[produk_key]

        # fact_stok
        cursor.execute("""
            INSERT INTO fact_stok (id_tanggal, id_produk, id_lokasi, stok, harga)
            VALUES (%s, %s, %s, %s, %s)
        """, (id_tanggal, id_produk, id_lokasi, row['stok'], row['harga']))

    conn.commit()
    cursor.close()
    conn.close()
    print("✅ Load ke star schema selesai!")


# ========== MAIN ==========
def etl_process_star():
    folder_path = r'C:\Users\DELL\Downloads\project_DWH'
    df_raw = extract(folder_path)
    df_clean = transform(df_raw)
    setup_star_schema()
    load_to_star(df_clean)

# Jalankan
etl_process_star()


✅ Extract selesai! Jumlah data: 8000 baris
✅ Transform selesai! Sisa data: 4000 baris
✅ Load ke star schema selesai!
